Cuda3
2000 이상 (적은 noise)된 eeg로 돌려보자.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from tqdm import tqdm
import os
import time
import nibabel as nib

import torch
from torch.utils.data import Dataset
import os
from pathlib import PosixPath
import nibabel as nib
from scipy.fft import fft
from scipy.signal import welch
import numpy as np
from torch.cuda.amp import autocast, GradScaler

from deeplearning_code_files.model import eegsubnet, mrisubnet, fusion
from deeplearning_code_files.train import Trainer
from deeplearning_code_files.datautils import MyTrainDataset, MyTrainDataset_sourcereconstructor
from deeplearning_code_files.utils import mri_downsampling,voxel_to_mask,ras_to_voxel

import numpy as np
import pandas as pd
import statistics
import random

import mne

In [4]:
device = torch.device('cuda:3')
mode = 'eeg'

In [16]:
ids =[21,22,23,24,25,26,27,28,29,30,31]
training_id = random.sample(ids, 8)
#[26, 13, 12, 27, 21, 28, 25, 11, 24, 23, 22, 3, 4, 9, 1, 7, 6, 10, 5, 31]
unseen_ids = [id for id in ids if id not in training_id]


print(f"seen:{training_id}")
print(f"unseen : {unseen_ids}")

seen:[24, 30, 26, 28, 31, 22, 23, 29]
unseen : [21, 25, 27]


In [17]:
training_data = MyTrainDataset(mri_id=training_id, 
                               outputtype='peak_ras',
                               eegtype='raw',
                               mri_n_downsampling=7,
                               eeg_per_mri=2000,
                                eeg_filter= 40,
                                ch_names= mne.channels.make_standard_montage("biosemi32").ch_names)

test_data = MyTrainDataset(mri_id=unseen_ids, 
                               outputtype='peak_ras',
                               eegtype='raw',
                               mri_n_downsampling=7,
                               eeg_per_mri=2000,
                                eeg_filter= 40,
                                ch_names= mne.channels.make_standard_montage("biosemi32").ch_names)

In [18]:

#Dataset size split
train_rate = 0.7
val_rate = 0.15
test_rate = 0.15
train_size = int(train_rate * len(training_data))
val_size = int(val_rate * len(training_data))
test_size = len(training_data) - train_size - val_size 
train_dataset, val_dataset, test_dataset = random_split(training_data, [train_size, val_size, test_size])
batch_size = 16
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [8]:
#When you want to see some data
def check_your_data(train_loader):
    for i, mri, eeg, dipoles in train_loader:
        i, checkmri,checkeeg,checksource = i, mri, eeg, dipoles
        break
    return i, checkmri, checkeeg, checksource

identity,mri,eeg,source = check_your_data(train_loader)


In [19]:

mid_dim = 100
mri_conv_dims = [1,1,1,1]
mri_mlp_dims=[2744,1000,200,mid_dim]
eeg_conv_dims=[32,32,32,16]
eeg_mlp_dims=[3104,1000,200,3]
fusion_dims = [2*mid_dim,mid_dim,10,3]


model = eegsubnet(conv_dims=eeg_conv_dims, mlp_dims=eeg_mlp_dims, conv_kernel_size=3, dropout=0).to(device)

In [10]:
class Trainer:
    def __init__(
            self,
            model : torch.nn.Module,
            train_data : DataLoader,
            validation_data : DataLoader,
            optimizer : torch.optim.Optimizer,
            gpu_id : id,
            save_energy: int,
            loss_ft,
            model_mode='mri+eeg',
            pinn_loss=False
            ) -> None:
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.save_energy = save_energy
        self.loss_ft = loss_ft
        self.train_loss_traj = []
        self.validation_loss_traj = []
        
        assert model_mode in ['mri+eeg', 'eeg'], 'model mode must be either "mri+eeg" or "eeg"'
        self.model_mode = model_mode
        
        self.pinnloss = pinn_loss
        
    def _run_batch(self, identity, mri, eeg, targets):
        if self.phase == 'training':
            self.optimizer.zero_grad()
            with autocast():                    
                if self.model_mode == 'eeg':
                    output = self.model(eeg)
                else:      
                    output = self.model([mri, eeg])
                if self.pinnloss == True:
                    brainmasks = torch.zeros_like(output)
                    for _,iii in enumerate(list(identity)):
                        brain_mask_path = PosixPath(f'/mnt/d/openneuro_mris/sub-{iii:02d}') 
                        brainmask = nib.load(os.path.join(brain_mask_path,'sample', 'mri','brainmask.mgz'))
                        brainmask = brainmask.get_fdata()
                        brainmask = torch.tensor(brainmask, dtype=output.dtype, device=output.device)
                        brainmasks[_] = brainmask
                    output = output * brainmasks
                    targets = targets * brainmasks
                    del brainmasks
                    loss = self.loss_ft(output, targets)
                else:
                    loss = self.loss_ft(output, targets)
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            self.running_train_loss += loss.item()
        if self.phase == 'validation':
            if self.model_mode == 'eeg':
                output = self.model(eeg)
            else:      
                output = self.model([mri, eeg])            
            loss = self.loss_ft(output, targets)
            self.running_validation_loss += loss.item()

    def _run_epoch(self, epoch):
        #training
        self.running_train_loss = 0
        self.model.train()
        self.phase = 'training'
        for identity, mri, eeg, targets in self.train_data:
            mri = mri.to(self.gpu_id)
            eeg = eeg.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            self._run_batch(identity, mri,eeg,targets)
        self.train_loss_traj.append(self.running_train_loss/len(self.train_data))
        
        #validation
        self.running_validation_loss = 0
        self.model.eval()
        self.phase = 'validation'
        with torch.no_grad():
            for identity, mri, eeg, targets in self.validation_data:
                mri = mri.to(self.gpu_id)
                eeg = eeg.to(self.gpu_id)
                targets = targets.to(self.gpu_id)
                self._run_batch(identity,mri,eeg,targets)
            self.validation_loss_traj.append(self.running_validation_loss/len(self.validation_data))
            
    def _save_checkpoint(self, epoch):
        ckp = self.model.state_dict()
        torch.save(ckp, f"checkpoint/{self.gpu_id}checkpoint.pt")
        if epoch % 10 == 1:
            print(f"Epoch {epoch} | Training checkpoint saved at checkpoit.pt")
        
    def train(self, max_epochs: int):
        self.scaler = GradScaler()

        for epoch in tqdm(range(max_epochs)):
            self._run_epoch(epoch)
            if epoch % self.save_energy == 0:
                self._save_checkpoint(epoch)
            print('EPOCH : %6d/%6d | Train Loss : %8.7f  | Validation : %8.7f ' %(epoch, max_epochs, self.train_loss_traj[epoch], self.validation_loss_traj[epoch]))
            torch.cuda.empty_cache()
            
    
    def test(self,
         test_data:DataLoader,
         loss_ft=None,
         ):
        import time
        import statistics
        self.phase = 'test'
        if loss_ft is None:
            loss_ft = self.loss_ft
        
        self.testloss = []
        self.inference_time = []
        
        self.model.eval()
        with torch.no_grad():
            for identity, mri, eeg, targets in test_data:
                mri = mri.to(self.gpu_id)
                eeg = eeg.to(self.gpu_id)
                targets = targets.to(self.gpu_id)
                if self.model_mode == 'mri+eeg':       
                    start_time = time.perf_counter()
                    output = self.model([mri, eeg])
                    end_time = time.perf_counter()
                elif self.model_mode == 'eeg':
                    start_time = time.perf_counter()
                    output = self.model(eeg)
                    end_time = time.perf_counter()
                elif self.model_mode == 'eeg fourier':
                    start_time = time.perf_counter()
                    output = self.model(eeg[0], eeg[1])
                    end_time = time.perf_counter()
                elif self.model_mode == 'mri+eeg+sensor':
                    start_time = time.perf_counter()
                    output = self.model([mri, eeg, sensor])
                    end_time = time.perf_counter()
                if self.pinnloss == True:
                    brainmasks = torch.zeros_like(output)
                    for _,iii in enumerate(list(identity)):
                        brain_mask_path = PosixPath(f'/mnt/d/openneuro_mris/sub-{iii:02d}') 
                        brainmask = nib.load(os.path.join(brain_mask_path,'sample', 'mri','brainmask.mgz'))
                        brainmask = brainmask.get_fdata()
                        brainmask = torch.tensor(brainmask, dtype=output.dtype, device=output.device)
                        brainmasks[_] = brainmask
                    output = output * brainmasks
                    targets = targets * brainmasks
                    del brainmasks
                    loss = self.loss_ft(output, targets)
                else:
                    loss = self.loss_ft(output, targets)
                self.testloss.append(loss.item())
                self.inference_time.append(end_time - start_time)
            self.testloss = np.array(self.testloss)
            ave_testloss = self.testloss.mean()
            print(f'mean testloss :{ave_testloss} (std: {statistics.stdev(self.testloss)})' )
            self.inference_time = np.array(self.inference_time)
            self.ave_inference_times= self.inference_time.mean()
            print(f'mean inference time : {self.ave_inference_times} and std : {statistics.stdev(self.inference_time)}')

In [20]:
trainer2 = Trainer(model=model,
                   train_data=train_loader,
                   validation_data=val_loader,
                   optimizer= torch.optim.Adam(model.parameters()),
                   gpu_id=device,
                   save_energy=50,
                   loss_ft=  nn.L1Loss(),
                   model_mode = mode
                   )

In [21]:
trainer2.train(500)

  0%|          | 1/500 [00:28<3:58:54, 28.73s/it]

EPOCH :      0/   500 | Train Loss : 29.1182347  | Validation : 29.0167002 


  0%|          | 2/500 [00:41<2:38:17, 19.07s/it]

EPOCH :      1/   500 | Train Loss : 29.0228803  | Validation : 28.9527757 


  1%|          | 3/500 [00:53<2:14:22, 16.22s/it]

...

EPOCH :    497/   500 | Train Loss : 1.0214107  | Validation : 9.8715683 


100%|█████████▉| 499/500 [2:10:37<00:13, 13.14s/it]

EPOCH :    498/   500 | Train Loss : 1.0039808  | Validation : 9.8885853 


100%|██████████| 500/500 [2:10:51<00:00, 15.70s/it]

EPOCH :    499/   500 | Train Loss : 1.0254375  | Validation : 9.8963243 


In [22]:
training_id

[24, 30, 26, 28, 31, 22, 23, 29]

In [23]:
trainer2.test(test_loader, loss_ft= nn.L1Loss())


mean testloss :9.878943996429443 (std: 1.9447128528667905)
mean inference time : 0.0008481933417109152 and std : 0.0003530145727554013


In [13]:
import pandas as pd

df = pd.DataFrame(trainer2.train_loss_traj)
df.to_excel("trainingloss.xlsx", index=False)

df = pd.DataFrame(trainer2.validation_loss_traj)
df.to_excel('validationloss.xlsx')



In [14]:
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=True)


In [15]:

df = pd.DataFrame(trainer2.testloss)
df.to_excel("testloss.xlsx", index=False)

mean testloss :8.179124071121215 (std: 1.4936965475933377)
mean inference time : 0.001135600646957755 and std : 0.001427135670976564


In [73]:
training_id = list(range(1,11))
training_data = MyTrainDataset(mri_id=training_id, 
                               outputtype='center_ras',
                               eegtype='fourier concatenate',
                               mri_n_downsampling=7,
                               eeg_per_mri=2000)

In [74]:

#Dataset size split
train_rate = 0.8
train_size = int(train_rate * len(training_data))
val_size = len(training_data) - train_size
train_dataset, val_dataset = random_split(training_data, [train_size, val_size])
batch_size = 16
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)    


In [75]:

#When you want to see some data
def check_your_data(train_loader):
    for i, mri, eeg, dipoles in train_loader:
        i, checkmri,checkeeg,checksource = i[0], mri[0], eeg[0], dipoles[0]
        break
    return i, checkmri, checkeeg, checksource

identity,mri,eeg,source = check_your_data(train_loader)


2 phases
1. MRI + EEG -> peak dipole coordinate
2. EEG + estimated peak dipole coordinate + pinn-based ->  Source Segmentation

phase 1. MRI + EEG -> peak dipole coordinate

In [76]:
from model import eegsubnet, mrisubnet, fusion

In [82]:
### MODEL SETUP
#Hyperparameters
mid_dim = 100
mri_conv_dims = [1,1,1,1]
mri_mlp_dims=[2744,1000,200,mid_dim]
eeg_conv_dims=[94,32,32,16]
eeg_mlp_dims=[3104,1000,200,3]
fusion_dims = [2*mid_dim,mid_dim,10,3]

model = eegsubnet(conv_dims=eeg_conv_dims, 
               mlp_dims=eeg_mlp_dims,
               conv_kernel_size=3, 
               dropout=0,
               ).to(torch.float32).to(device)

"""model = fusion(mri_conv_dims=mri_conv_dims, 
               mri_mlp_dims=mri_mlp_dims,
               mri_conv_kernel_size=(3,3,3), 
               eeg_conv_dims=eeg_conv_dims, 
               eeg_mlp_dims=eeg_mlp_dims,
               eeg_conv_kernel_size=3, 
               fusion_conv_dims=[1],
               fusion_dims=fusion_dims, 
               fusion_conv_kernel_size=3, 
               dropout=[0,0,0],
               batch_size=None, 
               output_as_3d=False
               ).to(torch.float32).to(device)
"""


'model = fusion(mri_conv_dims=mri_conv_dims, \n               mri_mlp_dims=mri_mlp_dims,\n               mri_conv_kernel_size=(3,3,3), \n               eeg_conv_dims=eeg_conv_dims, \n               eeg_mlp_dims=eeg_mlp_dims,\n               eeg_conv_kernel_size=3, \n               fusion_conv_dims=[1],\n               fusion_dims=fusion_dims, \n               fusion_conv_kernel_size=3, \n               dropout=[0,0,0],\n               batch_size=None, \n               output_as_3d=False\n               ).to(torch.float32).to(device)\n'

In [83]:
trainer1 = Trainer(model=model,
                   train_data=train_loader,
                   validation_data=val_loader,
                   optimizer= torch.optim.Adam(model.parameters()),
                   gpu_id=device,
                   save_energy=50,
                   loss_ft= nn.L1Loss(),
                   model_mode = 'eeg'
                   )

In [84]:
trainer1.train(500)

  0%|          | 0/500 [00:00<?, ?it/s]/home/pheeeeee/anaconda3/envs/eeg/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([16, 1, 3])) that is different to the input size (torch.Size([16, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  0%|          | 1/500 [00:32<4:33:15, 32.86s/it]

Epoch 0 | Training checkpoint saved at checkpoit.pt
EPOCH :      0/   500 | Train Loss : 27.0291240  | Validation : 26.8019404 


  0%|          | 2/500 [01:06<4:37:08, 33.39s/it]

EPOCH :      1/   500 | Train Loss : 26.8877990  | Validation : 26.8220860 


  1%|          | 3/500 [01:40<4:39:09, 33.70s/it]

...

EPOCH :    497/   500 | Train Loss : 26.7097558  | Validation : 26.7525835 


100%|█████████▉| 499/500 [4:59:42<00:36, 36.50s/it]

EPOCH :    498/   500 | Train Loss : 26.7151506  | Validation : 26.7329125 


100%|██████████| 500/500 [5:00:18<00:00, 36.04s/it]

EPOCH :    499/   500 | Train Loss : 26.7148394  | Validation : 26.7128815 


In [85]:
import pandas as pd

df = pd.DataFrame(trainer1.train_loss_traj)
df.to_excel("trainingloss.xlsx", index=False)

df = pd.DataFrame(trainer1.validation_loss_traj)
df.to_excel('validationloss.xlsx')


In [86]:
test_id = list(range(10,14))
test_data = MyTrainDataset(mri_id=test_id, 
                               outputtype='center_ras',
                               eegtype='fourier concatenate',
                               mri_n_downsampling=2,eeg_per_mri=2000)


test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle=True, drop_last=True)


In [87]:
trainer1.test(test_loader, loss_ft= nn.L1Loss())


mean testloss :26.847587074279787 (std: 1.9826123007004761)
mean inference time : 0.0012638593940064312 and std : 0.0012909374895521944


In [89]:
df = pd.DataFrame(trainer1.testloss)
df.to_excel("testloss.xlsx", index=False)

In [34]:
testloss = []
inference_time = []
loss_ft= nn.L1Loss()

model1.eval()
with torch.no_grad():
    for identity, mri, eeg, targets in test_loader:
        mri = mri.to(device)
        eeg = eeg.to(device)
        targets = targets.to(device)
        start_time = time.perf_counter()
        output = model1( eeg)
        end_time = time.perf_counter()
        loss = loss_ft(output, targets)
        testloss.append(loss.item())
        inference_time.append(end_time - start_time)
    testloss = np.array(testloss)
    ave_testloss = testloss.mean()
    print(f'mean testloss :{ave_testloss} (std: {statistics.stdev(testloss)})' )
    inference_time = np.array(inference_time)
    ave_inference_times= inference_time.mean()
    print(f'mean inference time : {ave_inference_times} and std : {statistics.stdev(inference_time)}')


mean testloss :5.106402254581451 (std: 1.0508110235600838)
mean inference time : 0.0009104147097095847 and std : 0.0006480245730479766


phase 2. EEG + MRI 로 나온 peak dipole과 EEG 이용하여 pinn토대로 source segmentation.

model 은 peak dipole(3d) + EEG --> segmentation in 3d mask volume 

In [205]:

for param in model1.parameters():
    param.requires_grad = False

In [ ]:
class source_reconstructor(nn.Module):
    def __init__(self, pretrained_model):
        super(source_reconstructor, self).__init__()
        self.pretrained_model = pretrained_model
        self.fc1 = nn.Linear(3, 128)  # Modify based on the output size
        self.fc2 = nn.Linear(128, num_classes)  # num_classes depends on your task

    def forward(self, x):
        x = self.pretrained_model(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:

# Instantiate and train the new model
new_model = NewModel(pretrained_model)

# Define optimizer and loss function
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop (simplified)
for epoch in range(num_epochs):
    outputs = new_model(new_data_inputs)
    loss = criterion(outputs, new_data_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()